In [9]:
import os
import pandas as pd
import tensorflow as tf 
import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [10]:
import cv2
import numpy as np
import pydicom

RESIZE_DIM = 256

def read_dcm(img_path):
    img_dicom = pydicom.dcmread(img_path)
    # convert to numpy array
    # Check the PhotometricInterpretation metadata
    if img_dicom.PhotometricInterpretation == 'MONOCHROME1':
        image = np.invert(img_dicom.pixel_array)
    else:
        image = img_dicom.pixel_array

    # Rescale the pixel values to [0, 255]
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Resize the image to desired size
    return cv2.resize(image, (RESIZE_DIM, RESIZE_DIM))

In [11]:
def get_train_and_test_data():
  print("Getting train and test data")
  df=pd.read_csv("/kaggle/input/rsna-bme548/RSNA_BME548/data.csv")

  files=glob.glob("/kaggle/input/rsna-bme548/RSNA_BME548/images/*.dcm")

  label=[]
  img_array=[]
  for i in files:
    filename = os.path.basename(i)
    img=read_dcm(i)
    img_array.append(np.array(img))
    # Get value of column "any" for the row with filename as index
    label.append(df.loc[df['filename'] == filename]['any'].values[0])

  x = np.array(img_array)
  x = x/255
  y = np.array(label)

  x = np.expand_dims(x,3)
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state = 0)
  
  return X_train, X_test, y_train, y_test

In [12]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters =filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,activation='relu',padding="same")
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [13]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [14]:
def get_model_accuracy(X_train, X_test, y_train, y_test):
  vgg = MyVGG(num_classes=2)
  vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  vgg.fit(X_train,y_train,epochs=5)

  # vgg.save_weights("models/vgg.weights.h5")

  y_pred=vgg.predict(X_test)

  y_pred_binary=[]
  for a in y_pred:
    y_pred_binary.append(np.argmax(a))

  y_pred_binary=np.array(y_pred_binary)

  accuracy = metrics.accuracy_score(y_test,y_pred_binary)

  print("Accuracy of model=",accuracy)

  print(classification_report(y_test, y_pred_binary, target_names=['0','1']))

  return accuracy

In [15]:
X_train, X_test, y_train, y_test = get_train_and_test_data()

Getting train and test data


In [16]:
get_model_accuracy(X_train, X_test, y_train, y_test)

Epoch 1/5


2024-04-24 06:44:41.990483: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65536: 1.05936, expected 0.681349
2024-04-24 06:44:41.990542: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65537: 2.21753, expected 1.83952
2024-04-24 06:44:41.990553: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65538: 2.5462, expected 2.16819
2024-04-24 06:44:41.990561: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65539: 2.07291, expected 1.6949
2024-04-24 06:44:41.990569: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65540: 2.56529, expected 2.18728
2024-04-24 06:44:41.990577: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65542: 2.19261, expected 1.8146
2024-04-24 06:44:41.990585: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 65543: 1.83463, expected 1.45662
2024-04-24 06:44:41.990593: E

  2/107 ━━━━━━━━━━━━━━━━━━━━ 8s 81ms/step - accuracy: 0.4609 - loss: 0.6949  

I0000 00:00:1713941103.005749     222 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


106/107 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5924 - loss: 0.6739

2024-04-24 06:45:11.700501: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.58222, expected 0.737369
2024-04-24 06:45:11.700565: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.41128, expected 1.56643
2024-04-24 06:45:11.700575: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.76015, expected 1.91529
2024-04-24 06:45:11.700583: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.49696, expected 1.65211
2024-04-24 06:45:11.700590: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.72918, expected 1.88432
2024-04-24 06:45:11.700598: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 3.22999, expected 2.38514
2024-04-24 06:45:11.700606: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.45164, expected 1.60678
2024-04-24 06:45:11.700613: E external/local_xla/xla/s

107/107 ━━━━━━━━━━━━━━━━━━━━ 41s 148ms/step - accuracy: 0.5930 - loss: 0.6735
Epoch 2/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.6687 - loss: 0.6144
Epoch 3/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.6983 - loss: 0.5879
Epoch 4/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 76ms/step - accuracy: 0.6913 - loss: 0.5803
Epoch 5/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.6807 - loss: 0.5874
18/19 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

2024-04-24 06:45:57.030748: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.81046, expected 0.976087
2024-04-24 06:45:57.030822: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.83784, expected 2.00347
2024-04-24 06:45:57.030832: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 3.1286, expected 2.29423
2024-04-24 06:45:57.030840: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.84179, expected 2.00742
2024-04-24 06:45:57.030847: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.97126, expected 2.13689
2024-04-24 06:45:57.030855: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 3.41114, expected 2.57677
2024-04-24 06:45:57.030863: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.89811, expected 2.06374
2024-04-24 06:45:57.030870: E external/local_xla/xla/se

19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step
Accuracy of model= 0.7
              precision    recall  f1-score   support

           0       0.66      0.82      0.73       301
           1       0.76      0.58      0.66       299

    accuracy                           0.70       600
   macro avg       0.71      0.70      0.70       600
weighted avg       0.71      0.70      0.70       600



0.7